In [1]:
import requests
import json
import pandas as pd
from sqlalchemy import create_engine
import pprint
pp = pprint.PrettyPrinter(indent=4)
import psycopg2 as pg
import pandas.io.sql as pd_sql
from copy import deepcopy
import numpy as np


In [2]:
# connecting to PostgrSQL to retrieve the data needed to make a full dataset for Tableau manipulations

In [3]:
# Twitch Client ID 
clientID = 'vb2kmh60pt0tee6o2c11ko6n2t1w9a'

In [4]:
# Postgres info to connect

connection_args = {
 "host": "twitchdata.chd4n5ul8muk.us-east-2.rds.amazonaws.com",
 "user": "postgres",
   "password":"FwwBFmleh65qYxKxDVb9",
 "port": 5432,
 "dbname": "twitchdata"
}

# We will talk about this magic Python trick!
connection = pg.connect(**connection_args)

In [5]:
#getting the genres and languages for all games to be able to filter the data

In [6]:
query = '''SELECT game_name, game_genres, language FROM general_view GROUP BY game_name, game_genres, language;
 '''
game_genre_language = pd_sql.read_sql(query, connection)

In [7]:
pwd

'/Users/jchow/Work/MetisCode/twitch-streamer-recommender/gitRoot/Twitch_Streamers_Recommender/Recommender_For_Existing_Streamers'

In [10]:
sully = pd.read_csv('../Jeremy_Notebooks/Natasha.csv', header = 0, names = ['date', 'game_name', 'hourly_channels', 'hourly_viewers'] )

In [30]:
import pickle

In [31]:
genres = pickle.load(open( "./Data/final_games (1).p", "rb"))

In [251]:
# making a dictionary of languages and genres to later attach to the data from sully gnome to be able to see trends filtered

In [35]:
genre_dict = pd.Series(genres.category.values,index=genres.game).to_dict()

In [36]:
sully['genres'] = sully['game_name'].map(genre_dict)

In [53]:
query = '''SELECT game_name, language FROM general_view group by game_name, language order by game_name;
 '''
lang = pd_sql.read_sql(query, connection)

In [68]:
games = set(lang['game_name'].to_list())

In [70]:
lang_dict = {}

for game in games:
    lang_dict[game] = lang[lang['game_name']==game]['language'].to_list()

In [71]:
lang_dict

{'Indiana Jones and the Fate of Atlantis': ['en', 'pt'],
 'Blade and Soul': ['zh', 'ko', 'ru', 'en'],
 'ToeJam & Earl: Back in the Groove!': ['en'],
 'Final Fantasy X': ['de', 'fr', 'en', 'zh'],
 'Vacation Simulator': ['en'],
 'PUBG MOBILE': ['vi',
  'ko',
  'de',
  'el',
  'zh-hk',
  'other',
  'es',
  'ro',
  'cs',
  'tr',
  'th',
  'id',
  'en',
  'sv',
  'uk',
  'ar',
  'nl',
  'zh',
  'it',
  'ru',
  'pt',
  'hu',
  'ja',
  'fr',
  'tl',
  'hi'],
 'Battlefield V': ['cs',
  'en',
  'bg',
  'zh',
  'ko',
  'other',
  'pl',
  'tr',
  'ru',
  'sk',
  'es',
  'it',
  'id',
  'el',
  'sv',
  'fi',
  'nl',
  'ro',
  'ar',
  'fr',
  'da',
  'pt',
  'no',
  'ja',
  'th',
  'hu',
  'de'],
 'Project Zomboid': ['tr', 'en'],
 'Call of Duty: WWII': ['en', 'fr', 'pt'],
 'Street Boyz': ['en'],
 'Contradiction: Spot The Liar!': ['en'],
 'Stellaris': ['hu',
  'sv',
  'de',
  'it',
  'ja',
  'es',
  'fr',
  'pt',
  'tr',
  'ru',
  'ko',
  'pl',
  'other',
  'zh',
  'ro',
  'en',
  'cs',
  'nl'],
 'Y

In [72]:
sully['languages'] = sully['game_name'].map(lang_dict)

In [74]:
sully = sully.dropna()

In [127]:
sully['genres'] = sully['genres'].apply(eval)

In [128]:
sully['languages'] = sully['languages'].apply(eval)

In [129]:
sully = sully.genres.apply(pd.Series) \
    .merge(sully, left_index = True, right_index = True) \
    .drop(["genres"], axis = 1)\
    .melt(id_vars = ['game_name', 'date', 'hourly_channels', 'hourly_viewers', 'languages'], value_name = "genres") \
    .drop("variable", axis = 1) \
    .dropna()

In [133]:
sully = sully.languages.apply(pd.Series) \
    .merge(sully, left_index = True, right_index = True) \
    .drop(["languages"], axis = 1)\
    .melt(id_vars = ['game_name', 'date', 'hourly_channels', 'hourly_viewers', 'genres'], value_name = "languages") \
    .drop("variable", axis = 1) \
    .dropna()

In [146]:
sully['channel_change'] = sully['hourly_channels'].diff()

In [147]:
sully['viewership_change'] = sully['hourly_viewers'].diff()

In [149]:
sully = sully.dropna()

In [216]:
sully_sorted = sully.sort_values(['game_name', 'genres', 'languages', 'date'], ascending=[True, True, True, True])


In [241]:
sully_sorted['channel_diff'] = sully_sorted.hourly_channels.diff(periods = 24)

In [245]:
sully_sorted['viewers_diff'] = sully_sorted.hourly_viewers.diff(periods = 24)

In [248]:
#sully_sorted.to_csv('sully_sorted.csv', index = False)